In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

# test =pd.read_csv("/content/gdrive/MyDrive/adultData/MLtest_notStandard.csv")
# olddata = pd.read_csv("/content/gdrive/MyDrive/adultData/adult-all.csv")

In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/adultData/MLdata_notStandard.csv")
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
import csv
from itertools import islice
with open('/content/gdrive/MyDrive/adultData/MLdata_notStandard.csv') as file_obj:
  reader_obj = csv.reader(file_obj)
  data_list = []
  for row in islice(reader_obj, 1, 300000):
    numeric = [0,2,4,10,11,12]
    for i in numeric:
      row[i] = int(row[i]) 
   
    data_list.append(row)
print(len(data_list))
print(data_list[0])


30107
[39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States', '<=50K']


In [ ]:

with open('/content/gdrive/MyDrive/adultData/MLtest_notStandard.csv') as file_obj:
  reader_obj = csv.reader(file_obj)
  test_list = []
  for row in islice(reader_obj, 1, 50000):
    numeric = [0,2,4,10,12]
    for i in numeric:
      row[i] = int(row[i])
    row[11] = float(row[i])
    test_list.append(row)
len(test_list)
print(test_list[0])


[25, 'Private', 226802, '11th', 7, 'Never-married', 'Machine-op-inspct', 'Own-child', 'Black', 'Male', 0, 40.0, 40, 'United-States', '<=50K.']


In [ ]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [ ]:

data_col = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week' , 'native-country',
       'Salary']

In [ ]:
from pandas.core.arrays import numeric
class Question:
  def __init__(self, column, value):
    self.column = column
    self.value = value
  def match(self, example):
    val = example[self.column]
    if is_numeric(val):
      return val >= self.value
    else:
      return val == self.value
  def __repr__(self):
    condition = "=="
    if is_numeric(self.value):
            condition = ">="
    return "Is %s %s %s" % (data_col[self.column], condition, str(self.value))

In [ ]:
from __future__ import print_function

In [ ]:
import csv
from itertools import islice

def partition(data,question):
  
  # Partitions a dataset.

  # For each row in the dataset, check if it matches the question. If true
  # so, add it to 'true rows', otherwise, add it to 'false rows'.
  val= question.value
  col = question.column
  true_rows, false_rows = [], []
  
  
  for row in data:
    row_ = row[:]
    if question.match(row):
      true_rows.append(row_)
    else:
      false_rows.append(row_)
     
  return true_rows, false_rows

In [ ]:
# t,f =partition(data_list,Question(5,'Married-civ-spouse'))
# f,k =partition(data_list,Question(5,'Never-married'))
# data_list[0]


In [ ]:
def class_counts(rows):
    # Counts the number of each type of example in a dataset.
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        #  the label is the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [ ]:
print ( "Predict", class_counts(data_list))

Predict {' <=50K': 213, ' >50K': 62}


In [ ]:
def entropy(y):
  counts = class_counts(y)
  entropy = 0
  for lbl in counts:
    a = counts.get(lbl) / float(len(y))
    entropy -= a*np.log2(a)
  return(entropy)


In [ ]:
def information_gain(true_, false_, current_uncertainty):
  p = float(len(true_)) / (len(true_) + len(false_))
  return current_uncertainty - p * entropy(true_) - (1 - p) * entropy(false_)

In [ ]:
# information_gain(t, f, entropy(data_list))

In [ ]:
def best_split(data):
  best_gain = 0
  best_question = None
  true_rows = []
  false_rows=[]
  column =0 
  value = None
  current_uncertainty = entropy(data)
  n_features = [4,5,10,12]
  for col in n_features:
    values = set([row[col]for row in data])
    for val in values:
      question = Question(col, val)
      true_Rows, false_Rows = partition(data, question)
      if len(true_Rows) == 0 or len(false_Rows) == 0:
        continue
      info_gain = information_gain(true_Rows, false_Rows, current_uncertainty)
      if info_gain >= best_gain:
        best_gain, best_question,true_rows, false_rows, column, value= info_gain, question, true_Rows.copy(), false_Rows.copy(),col,val
  return {'best_gain': best_gain,'best_question': best_question,'true_rows':true_rows,'false_rows': false_rows,'question_column' :column,'question_value': value}  


In [ ]:
def to_terminal(group):
  outcomes = [row[-1] for row in group]
  return max(set(outcomes), key=outcomes.count)

In [ ]:
root = best_split(data_list)
print(root['question_column'])
print(root['best_question'])
print(root['best_gain'])
print(len(root['true_rows']))
root['true_rows']
print(root['question_value'])  

5
Is marital-status == Married-civ-spouse
0.1560078311019322
129
Married-civ-spouse


In [ ]:
print ( "Predict", class_counts(data_list))

Predict {'<=50K': 23438, '>50K': 6668}


In [ ]:
def build_sub_tree(node, max_depth, min_size,depth):
  true_,false_ = node['true_rows'],node['false_rows']
  del(node['true_rows'])
  del(node['false_rows'])
  if not true_ or not false_:
    node['true_rows'] = node['false_rows'] = to_terminal(true_ + false_)
    return
  if depth >= max_depth:
    node['true_rows'], node['false_rows'] = to_terminal(true_), to_terminal(false_)
    return
  if len(true_) <= min_size:
    node['true_rows'] = to_terminal(true_)  
  else:
    node['true_rows'] = best_split(true_)
    # data_col.remove(data_col[node['true_rows']['question_column']])
    build_sub_tree(node['true_rows'], max_depth, min_size,depth+1)
  if len(false_) <= min_size:
    node['false_rows'] = to_terminal(false_)  
  else:
    node['false_rows'] = best_split(false_)
    build_sub_tree(node['false_rows'], max_depth, min_size,depth+1)
    # data_col.remove(data_col[node['false_rows']['question_column']])
 

In [ ]:
def build_tree(data,max_depth,min_size):
  root = best_split(data)
  build_sub_tree(root, max_depth,min_size,1)
  return root

In [ ]:
print(a)

{'best_gain': 0.15239823241974715, 'best_question': Is marital-status ==  Married-civ-spouse, 'question_column': 5, 'question_value': ' Married-civ-spouse', 'true_rows': ' >50K', 'false_rows': {'best_gain': 0.1467899686612929, 'best_question': Is marital-status ==  Married-civ-spouse, 'question_column': 5, 'question_value': ' Married-civ-spouse', 'true_rows': ' <=50K', 'false_rows': {'best_gain': 0.14887855858310464, 'best_question': Is marital-status ==  Married-civ-spouse, 'question_column': 5, 'question_value': ' Married-civ-spouse', 'true_rows': ' <=50K', 'false_rows': {'best_gain': 0.14839528793359683, 'best_question': Is marital-status ==  Married-civ-spouse, 'question_column': 5, 'question_value': ' Married-civ-spouse', 'true_rows': ' <=50K', 'false_rows': {'best_gain': 0.15322771335233853, 'best_question': Is marital-status ==  Married-civ-spouse, 'question_column': 5, 'question_value': ' Married-civ-spouse', 'true_rows': ' >50K', 'false_rows': ' <=50K'}}}}}


In [ ]:
def print_tree(node,spacing=""):
  if isinstance(node, dict):
    print(spacing +str(node['best_question']))

    print (spacing + '--> True_rows:')
    print_tree(node['true_rows'],spacing+"    ")

    print (spacing + '--> False_rows:')
    print_tree(node['false_rows'],spacing+"    ")
  else:
    print(spacing + "Predict",node)

In [ ]:
def predict(node,row):
  numeric = [0,2,4,10,11,12]
  if node['question_column'] in numeric:
    if int(row[node['question_column']]) > node['question_value']:
      if isinstance(node['true_rows'], dict):
        return predict(node['true_rows'],row)
      else:
        return node['true_rows']
    else:
      if isinstance(node['false_rows'], dict):
        return predict(node['false_rows'],row)
      else:
        return node['false_rows']
  else:
    if row[node['question_column']] ==  node['question_value']:
      if isinstance(node['true_rows'], dict):
        return predict(node['true_rows'],row)
      else:
        return node['true_rows']
    else:
      if isinstance(node['false_rows'], dict):
        return predict(node['false_rows'],row)
      else:
        return node['false_rows']



In [ ]:
t =[23, 'Private', 82497, 'Some-college', 10, 'Never-married', 'Sales', 'Own-child', 'White', 'Female', 0, 0, 28, 'United-States', '<=50K']

In [ ]:
predict(a,t)

'<=50K'

In [ ]:
testing_data = [60,
 'Local-gov',
 141637,
 'Masters',
 14,
 'Married-civ-spouse',
 'Prof-specialty',
 'Husband',
 'White',
 'Male',
 0,
 0,
 40,
 'United-States',
 '>50K']


In [ ]:
predict(a,testing_data)

'>50K'

In [ ]:
from random import randrange
def cross_validation_split(data, n_folds):
	dataset_split = []
	dataset_copy = data.copy()
	fold_size = int(len(data) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split 

In [ ]:
folds = cross_validation_split(data_list, 5)
scores = list()
for fold in folds:
  train_set = list(folds)
  train_set.remove(fold)
  train_set = sum(train_set, [])
  a = build_tree(train_set,7,200)
  test_set = list()
  for row in fold:
    row_copy = row[:]
    test_set.append(row_copy)
    # row_copy[-1] = None
  accuracy = accuracy_trainData(test_set,a)
  scores.append(accuracy)
# b =sum(scores)/float(len(scores)


In [ ]:
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Mean Accuracy: 83.342%


In [ ]:
def accuracy_metric(data,loaded_model):
  correct = 0
  predictions = list()
  actual = [row[-1] for row in data]
  for row in data:
    prediction = predict(loaded_model, row)
    predictions.append(prediction)
  for i in range(len(actual)):
    # print(actual[i])
    # print(predictions[i])
    if actual[i] == predictions[i]+".":
      correct += 1
  return correct / float(len(actual)) * 100.0

In [ ]:
def accuracy_trainData(data,loaded_model):
  correct = 0
  predictions = list()
  actual = [row[-1] for row in data]
  for row in data:
    prediction = predict(loaded_model, row)
    predictions.append(prediction)
  for i in range(len(actual)):
    # print(actual[i])
    # print(predictions[i])
    if actual[i] == predictions[i]:
      correct += 1
  return correct / float(len(actual)) * 100.0

In [ ]:
print_tree(a)

Is marital-status == Married-civ-spouse
--> True_rows:
    Is capital-gain >= 5178
    --> True_rows:
        Predict >50K
    --> False_rows:
        Is education-num >= 11
        --> True_rows:
            Is age >= 29
            --> True_rows:
                Is workclass == Self-emp-not-inc
                --> True_rows:
                    Predict <=50K
                --> False_rows:
                    Is capital-gain >= 3411
                    --> True_rows:
                        Predict <=50K
                    --> False_rows:
                        Is education-num >= 16
                        --> True_rows:
                            Predict >50K
                        --> False_rows:
                            Predict >50K
            --> False_rows:
                Predict <=50K
        --> False_rows:
            Is education-num >= 9
            --> True_rows:
                Is age >= 30
                --> True_rows:
                    Is hours-per-week >= 

In [ ]:
print_tree(a)

Is marital-status == Married-civ-spouse
--> True_rows:
    Is capital-gain >= 5178
    --> True_rows:
        Predict >50K
    --> False_rows:
        Is education-num >= 11
        --> True_rows:
            Is age >= 29
            --> True_rows:
                Is workclass == Self-emp-not-inc
                --> True_rows:
                    Predict <=50K
                --> False_rows:
                    Is capital-gain >= 3411
                    --> True_rows:
                        Predict <=50K
                    --> False_rows:
                        Is education-num >= 16
                        --> True_rows:
                            Predict >50K
                        --> False_rows:
                            Is occupation == Exec-managerial
                            --> True_rows:
                                Predict >50K
                            --> False_rows:
                                Is fnlwgt >= 29841
                                --> True_

In [ ]:
accuracy_trainData(data_list,a)

84.26142047349117

In [ ]:
a =build_tree(data_list,7,200)

In [ ]:
accuracy_metric(test_list,a)

82.25538971807629

In [ ]:
import pickle
filename ='/content/gdrive/MyDrive/adultData/decisionTree_Model7 .sav'
pickle.dump(a, open(filename, 'wb'))

In [ ]:
import pickle
filename ='/content/gdrive/MyDrive/adultData/decisionTree_Model .sav'
loaded_model = pickle.load(open(filename, 'rb'))
